In [1]:
import tensorflow as tf
import numpy as np
from layers import architecture, evaluate
from utils.cifar_utils import load_data

In [2]:
# Load the raw CIFAR-10 data.
X_train, y_train = load_data(mode='train')
X_test, y_test = load_data(mode='test')

# Data organizations:
# Train data: 49000 samples from original train set: 1~49000
# Validation data: 1000 samples from original train set: 49000~50000
num_training = 45000
num_validation = 5000

X_val = X_train[-num_validation:, :]
y_val = y_train[-num_validation:]

X_train = X_train[:num_training, :]
y_train = y_train[:num_training]

./data/cifar-10-python.tar.gz already exists. Begin extracting...
./data/cifar-10-python.tar.gz already exists. Begin extracting...


In [3]:
# # Preprocessing: subtract the mean value across every dimension for training data, and reshape it to be RGB size
# mean_image = np.mean(X_train, axis=0)
# std_image = np.std(X_train, axis=0)

# X_train = (X_train.astype(np.float32) - mean_image.astype(np.float32)) / std_image
# X_val = (X_val.astype(np.float32) - mean_image) / std_image
# X_test = (X_test.astype(np.float32) - mean_image) / std_image

In [4]:
X_train = X_train.reshape([X_train.shape[0],3,32,32]).transpose((0,2,3,1))
X_val = X_val.reshape([X_val.shape[0],3,32,32]).transpose((0,2,3,1))
X_test = X_test.reshape([X_test.shape[0],3,32,32]).transpose((0,2,3,1))

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape', X_test.shape)
print('Test labels shape', y_test.shape)

Train data shape:  (45000, 32, 32, 3)
Train labels shape:  (45000,)
Validation data shape:  (5000, 32, 32, 3)
Validation labels shape:  (5000,)
Test data shape (10000, 32, 32, 3)
Test labels shape (10000,)


# change ADAM?
# Use better initializations for batch_norm?
# add model saver/checkpointer. In addition keep track of best validation error and save that model

In [5]:
inputs = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3])
random_rolls = tf.placeholder(dtype=tf.float32, shape=[54])
y_inputs = tf.placeholder(dtype=tf.uint8, shape=[None])
is_training = tf.placeholder(dtype=tf.bool)

lr = 0.1
weight_decay = 1e-4
momentum = 0.9
batch_size = 128
train_size = 45000
epochs = 500

out = architecture(inputs, random_rolls, is_training)
labels = tf.one_hot(y_inputs, 10)
softmax_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=labels))
# confirm this is right
regularizer_loss = weight_decay * tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables('stoch_depth')
                                            if 'weights' in var.name])
loss = softmax_loss + regularizer_loss
training_summary = tf.summary.scalar('train_loss', loss)
step = tf.train.MomentumOptimizer(lr, momentum, use_nesterov=True).minimize(loss)
# step = tf.train.AdamOptimizer(lr).minimize(loss)
eve = evaluate(out, y_val)
validation_summary = tf.summary.scalar('Stoch_error', 100 * eve / len(y_val))

In [6]:
# from datetime import datetime
# now = datetime.now()
# logdir = "tf_logs/" + now.strftime("%Y%m%d-%H%M%S") + "/"

# Tests (ignore)

In [7]:
# confirm tensorboard
random_rolls_batch = np.random.uniform(size=54)
X_train_batch = X_train[list(range(20))]
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(out, feed_dict={inputs: X_train_batch,
                             random_rolls: random_rolls_batch,
                             is_training: True})
    sess.run(out, feed_dict={inputs: X_val,
                             random_rolls: random_rolls_batch,
                             is_training: False})
    writer.close()

In [8]:
# # confirm training = False gives same answers
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, False)
#             if i == 0:
#                 sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))

In [9]:
# # confirm training = False but new init gives diff answers
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, False)
#             sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))

In [10]:
# confirm training = True gives diff answers
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, True)
#             if i == 0:
#                 sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))